In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import datetime
import time
#import psycopg2
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive_output
#from IPython.display import display, HTML

from plotly.subplots import make_subplots
import plotly.graph_objects as go

import qgrid
import random

еще сделать:

    в banks добавить время обслуживания с-по.

In [51]:
amount_start = 100.0
amount_end = 2000.0
amount_step = 100.0
fix = 5.0
percent = 1.5

style = {}
#style = {'max_width': '10%'}
#layout=Layout(width='50%', height='80px')

w_amount_name  = widgets.Label('Сумма операции', style=style)
w_amount_start = widgets.FloatText(value=amount_start, description='START:', disabled=False, style=style)
w_amount_end   = widgets.FloatText(value=amount_end, description='END:', disabled=False, style=style)
w_amount_step = widgets.FloatText(value=amount_step, description='STEP:', disabled=False, style=style)
w_fix = widgets.FloatText(value=fix, description='FIX:', disabled=False, style=style)
w_percent = widgets.FloatText(value=percent, description='PERCENT:', disabled=False, style=style)
w_vbox_amount = widgets.VBox([w_amount_name, w_amount_start, w_amount_end, w_amount_step, w_fix, w_percent])


banks = [
    {'name': 'BANK1', 'fix': 10.0, 'percent': 1.0, 'min': 0.0, 'max': 10000.0},
    {'name': 'BANK2', 'fix': 0.0, 'percent': 2.0, 'min': 0.0, 'max': 10000.0},
    {'name': 'BANK3', 'fix': 0.0, 'percent': 1.6, 'min': 10.0, 'max': 2000.0}
]

banks_lst = [w_vbox_amount]
for bank in banks:
    w_vbox = widgets.VBox([
        widgets.Label(bank['name']),
        widgets.FloatText(description='fix:', value=bank['fix']),
        widgets.FloatText(description='percent:', value=bank['percent']),
        widgets.FloatText(description='min:', value=bank['min']),
        widgets.FloatText(description='max:', value=bank['max']),
    ])
    banks_lst.append(w_vbox)

In [52]:
w = widgets.HBox(banks_lst)

'''
client_label = widgets.Label(value='0')
def handle_change(change):
    client_label.value = change.new
w.observe(handle_change, names='value')
'''

kwargs = {}
for k,v in enumerate(w.children):
    #print(k,v)
    for x,y in enumerate(v.children):
        #print(x,y)
        kwargs['{}_{}'.format(k,x)] = y

#print(kwargs)

df_external = pd.DataFrame()

        
def myfunc(**kwargs):
    global df_external
    banks = []
    bank_dict = {
        0: 'name',
        1: 'fix',
        2: 'percent',
        3: 'min',
        4: 'max'
    }
    #print(banks)
    for k,v in kwargs.items():
        #print(k,v)
        x,y = int(k.split('_')[0]), int(k.split('_')[1])
        #print(x,y)
        if len(banks)<x:
            banks.append({})
        if x>0:
            banks[x-1][bank_dict[y]]=v
    #print(banks)
        
    start = kwargs['0_1']
    end = kwargs['0_2']
    step = kwargs['0_3']
    myfix = kwargs['0_4']
    mypercent = kwargs['0_5']
    
    bank_cols = [bank['name'] for bank in banks]
    df = pd.DataFrame(columns = ['x']+bank_cols+['MIN', 'DELTA'])
    
    for x in range(int(start), int(end), int(step)):
        row = [x]
        for bank in banks:
            fix = bank['fix']
            percent = bank['percent']*x/100
            full = fix + percent
            if full<bank['min']:
                result = bank['min']
            elif full>bank['max']:
                result = bank['max']
            else:
                result = full
            row.append(result)
        
        minbank = min(row[1:])
        idx = row.index(minbank)-1
        row.append(bank_cols[idx])
        row.append(myfix+mypercent*x/100-minbank)
        df.loc[len(df)] = row

    #df_external = df.copy()
    '''
    fig = go.Figure()
    for col in df.columns:
        #if col!='x':  
        fig.add_trace(go.Scatter(x=df['x'], y=df[col], mode='lines', name=col))
    fig.update_layout(height=500, title_text="Bank tariffs", showlegend=False)
    fig.show()
    '''
    plt.figure()
    df.plot(x='x', y=bank_cols+['DELTA'], subplots=False, figsize=(20, 5))
    #client_label.value = str(random.random())
    plt.show()
    display(df)

In [53]:
out = interactive_output(myfunc, kwargs)
display(w, out)

Output()